In [1]:
from io import StringIO
import numpy as np
import polars as pl
import pandas as pd
from subprocess import call
from gtfparse import read_gtf

### Set parameters

In [2]:
exon_flank_nt = 5 # flanking nucleotides from the start and end of exons
number_of_threads = 4 # number of threads used in bcftools output compression
tag_str = 'Vineel_102323' # DNAnexus job tag

project_path = 'project-GGy3Bb0JqBj7zfxY8v4by61X:/'

# input vcf path (end with '/')
dx_vcf_path = project_path + "Bulk/Exome\ sequences/Population\ level\ exome\ OQFE\ variants,\ pVCF\ format\ -\ final\ release/"
# output vcf path (end with '/')
dx_vcf_out_path = project_path + "Tian_folder/Vineel_genes_102323/"
# resource path (end with '/')
dx_resource_path = project_path + "GRCh38_resources/"
# difficult regions bed filename
diff_bed = 'GRCh38_alldifficultregions.bed.gz'
# reference genome filename
ref_genome = 'GRCh38_reference_genome.fa' # index filename is inferred

!dx mkdir -p {dx_vcf_out_path} # create gene.vcf output folder

### Helper functions

In [3]:
def get_overlapping_UKB_vcfs(df_gene, df_blk, diagnosis=False):
    vcf_prefix = 'ukb23157_'
    vcf_suffix = '_v1.vcf.gz'
    
    all_vcf_files = []
    df_gene = df_gene.assign(vcf_files = '')
    for index, row in df_gene.iterrows():
        new_vcf_files = df_blk.loc[(df_blk['seqname'] == row['seqname']) & 
                                   ((df_blk['end_pos'] >= row['exon_flank_start']) & (df_blk['start_pos'] <= row['exon_flank_end'])),
                                   'chr_blk_str'].tolist()
        new_vcf_files = list(vcf_prefix + x + vcf_suffix for x in set(new_vcf_files))
        df_gene.loc[index,'vcf_files'] = ','.join(new_vcf_files)
        all_vcf_files = all_vcf_files + new_vcf_files
    all_vcf_files = list(set(all_vcf_files))
    
    if diagnosis:
        return(df_gene)
    else:
        return(all_vcf_files)

### List of gene symbols as input

In [4]:
genes = ["PALB2"]

# # to read from txt:
# with open("resources/my_gene_file.txt", "r") as gene_file:
#     genes = gene_file.readlines()
#     genes = [l.replace("\n", "") for l in genes]
#     print(genes)

### Load pVCF block coordinates

In [5]:
df_blk = pd.read_table("./resources/pvcf_blocks.txt", sep = '\t', names = ['ind', 'chr', 'blk', 'start_pos', 'end_pos'])
df_blk['seqname'] = 'chr' + df_blk['chr'].map(str)
df_blk.loc[df_blk['seqname'] == 'chr23', 'seqname'] = 'chrX'
df_blk.loc[df_blk['seqname'] == 'chr24', 'seqname'] = 'chrY'
df_blk['chr_blk_str'] = df_blk['seqname'].str.replace('chr', 'c') + '_b' + df_blk['blk'].map(str)
df_blk

ind  chr  blk  start_pos    end_pos seqname chr_blk_str
0      1    1    0          1    1218130    chr1       c1_b0
1      2    1    1    1218131    1426969    chr1       c1_b1
2      3    1    2    1426970    1758871    chr1       c1_b2
3      4    1    3    1758872    2514221    chr1       c1_b3
4      5    1    4    2514222    3782130    chr1       c1_b4
..   ...  ...  ...        ...        ...     ...         ...
972  973   23   20  135552245  141897932    chrX      cX_b20
973  974   23   21  141897933  152168662    chrX      cX_b21
974  975   23   22  152168663  153788223    chrX      cX_b22
975  976   23   23  153788224  156040895    chrX      cX_b23
976  977   24    0          1   57227415    chrY       cY_b0

[977 rows x 7 columns]

### Load MANE transcript coordinates

In [6]:
df = pd.read_csv("./resources/MANE.GRCh38.v1.0.select_ensembl_genomic.csv.gz")
df = df.loc[(df['feature'] == 'exon') & (df['gene_name'].isin(genes))]
df = df[['seqname', 'start', 'end', 'gene_name']]
df['exon_flank_start'] = df['start'] - exon_flank_nt
df['exon_flank_end'] = df['end'] + exon_flank_nt
df['region'] = ((df['seqname'] + ':').str.cat(df['exon_flank_start'].astype(str)) + '-').str.cat(df['exon_flank_end'].astype(str))
df

seqname     start       end gene_name  exon_flank_start   
397668   chr16  23641110  23641310     PALB2          23641105  \
397671   chr16  23638070  23638129     PALB2          23638065   
397673   chr16  23637850  23637952     PALB2          23637845   
397675   chr16  23634862  23636334     PALB2          23634857   
397677   chr16  23629640  23630469     PALB2          23629635   
397679   chr16  23629204  23629275     PALB2          23629199   
397681   chr16  23626236  23626397     PALB2          23626231   
397683   chr16  23624009  23624094     PALB2          23624004   
397685   chr16  23622969  23623130     PALB2          23622964   
397687   chr16  23621362  23621478     PALB2          23621357   
397689   chr16  23614004  23614091     PALB2          23613999   
397691   chr16  23607864  23608012     PALB2          23607859   
397693   chr16  23603165  23603669     PALB2          23603160   

        exon_flank_end                   region  
397668        23641315  chr16:23641105-23641315  
397671        23638134  chr16:23638065-23638134  
397673        23637957  chr16:23637845-23637957  
397675        23636339  chr16:23634857-23636339  
397677        23630474  chr16:23629635-23630474  
397679        23629280  chr16:23629199-23629280  
397681        23626402  chr16:23626231-23626402  
397683        23624099  chr16:23624004-23624099  
397685        23623135  chr16:23622964-23623135  
397687        23621483  chr16:23621357-23621483  
397689        23614096  chr16:23613999-23614096  
397691        23608017  chr16:23607859-23608017  
397693        23603674  chr16:23603160-23603674

### Run Swiss-army-knife on DNAnexus
- get region info for each gene from block file
- bcftools command for step 2 finished
- bcftools command for step 3 & 4 TBD
- list genes not included in MANE set

In [7]:
genes_not_found = []
genes_found = []
known_large_genes = ["DSP", "TSC2", "TTN", "NCOA3"]

for gene in genes:
    df_gene = df.loc[df['gene_name'] == gene]
    if df_gene.shape[0] > 0:
        genes_found.append(gene)
        
        vcf_files = get_overlapping_UKB_vcfs(df_gene, df_blk)
        vcf_str = ' '.join(vcf_files)
        region_str = ','.join(df_gene['region'].to_list())
        
        if ((len(vcf_files) > 1) or (gene in known_large_genes)):
            mem_level = "mem2_ssd1_v2_x16" # dynamically change memory level when submitting jobs on DNAnexus
        else:
            mem_level = "mem1_ssd1_v2_x4"
        
        # filtering variants
        bcftools_cmd1 = "bcftools concat -Ou -a -r " + region_str + " " + vcf_str
        bcftools_cmd2 = "bcftools view -Ou --max-alleles 5 -T ^" + diff_bed
        bcftools_cmd3 = "bcftools +fill-tags -Ou -- -t all"
        bcftools_cmd4 = "bcftools norm -Ou -m - -f " + ref_genome 
        bcftools_cmd5 = "bcftools view -Oz -i 'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(DP>=10 & GT!=\\\"mis\\\")> 0.9' > " + gene + "_variants.vcf.gz"
        
        # extract carriers
        bcftools_cmd6 = "mkdir -p carrier_out"
        bcftools_cmd7 = "bcftools query -i 'GT=\\\"RA\\\"|GT=\\\"AR\\\"|GT=\\\"AA\\\"' -f '%CHROM  %POS %REF %ALT %INFO/AF [%SAMPLE|]\n' " + gene + "_variants.vcf.gz > carrier_out/" + gene + ".ssv"
        
        # parsing command
        bcftools_command_a = " | ".join([bcftools_cmd1, bcftools_cmd2, bcftools_cmd3, bcftools_cmd4, bcftools_cmd5])
        bcftools_command_b = " && ".join([bcftools_cmd6, bcftools_cmd7])
        bcftools_command = bcftools_command_a + " && " + bcftools_command_b
        
        # parsing input
        dx_input_str = ' '.join(set('-iin="' + dx_vcf_path + x + '"' for x in vcf_files).union(set('-iin="' + dx_vcf_path + x + '.tbi"' for x in vcf_files)))
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + diff_bed + '"'
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '"'
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '.fai"'
        
        # final dx command
        dx_command = 'dx run swiss-army-knife --instance-type ' + mem_level + ' -y --brief ' + dx_input_str + ' -icmd="' + bcftools_command + '" --destination ' + dx_vcf_out_path + ' --tag "' + tag_str + '" --property gene=' + gene
        !{dx_command}
    else:
        genes_not_found.append(gene)
            
print('Genes not found in MANE database:')
print(genes_not_found)

job-Gb3GVf8JqBjKX5bx1zpVj221
Genes not found in MANE database:
[]


In [12]:
dx_command_c

'dx run swiss-army-knife --instance-type mem2_ssd1_v2_x16 -y --brief -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c7_b4_v1.vcf.gz.tbi" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c7_b3_v1.vcf.gz.tbi" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c7_b3_v1.vcf.gz" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c7_b4_v1.vcf.gz" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resources/GRCh38_alldifficultregions.bed.gz" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resources/GRCh38_reference_genome.fa" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resourc

### Below is for testing purposes

In [53]:
gene = 'APOB'
df_gene = df.filter(pl.col('gene_name') == gene)
vcf_files = get_overlapping_UKB_vcfs(df_gene, df_blk)
vcf_str = ' '.join(vcf_files)
region_str = ','.join(df_gene['region'].to_list())
mem_level = len(vcf_files)

bcftools_cmd1 = "bcftools concat -Ou -a -r " + region_str + " " + vcf_str
bcftools_cmd2 = "bcftools view -Ou -T ^" + diff_bed
bcftools_cmd3 = "bcftools norm -Ou -m - -f " + ref_genome
bcftools_cmd4 = "bcftools annotate -Ou --set-id '%CHROM\_%POS\_%REF\_%ALT'"
bcftools_cmd5 = "bcftools view -Oz --threads " + str(number_of_threads) + " -i 'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(DP>=10 & GT!=\\\"mis\\\")> 0.9' > " + gene + ".vcf.gz"
bcftools_command = " | ".join([bcftools_cmd1, bcftools_cmd2, bcftools_cmd3, bcftools_cmd4, bcftools_cmd5])

dx_input_str = ' '.join(set('-iin="' + dx_vcf_path + x + '"' for x in vcf_files).union(set('-iin="' + dx_vcf_path + x + '.tbi"' for x in vcf_files)))
dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + diff_bed + '"'
dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '"'
dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '.fai"'

dx_command = 'dx run swiss-army-knife --instance-type mem' + str(mem_level) + '_ssd1_v2_x4 -y --brief ' + dx_input_str + ' -icmd="' + bcftools_command + '" --destination ' + dx_vcf_out_path + ' --tag "' + tag_str + '" --property gene=' + gene

print(bcftools_command)
print(dx_command)



bcftools concat -Ou -a -r chr2:21043859-21044078,chr2:21043508-21043556,chr2:21042356-21042481,chr2:21040933-21041088,chr2:21037953-21038116,chr2:21037095-21037260,chr2:21035579-21035713,chr2:21034811-21034906,chr2:21033294-21033523,chr2:21032349-21032586,chr2:21029893-21030020,chr2:21029634-21029790,chr2:21028322-21028543,chr2:21027823-21028070,chr2:21026783-21026969,chr2:21024928-21025129,chr2:21023520-21023697,chr2:21022826-21023047,chr2:21019718-21019910,chr2:21018987-21019118,chr2:21016434-21016654,chr2:21015365-21015550,chr2:21015068-21015265,chr2:21014443-21014598,chr2:21013155-21013538,chr2:21005075-21012656,chr2:21004556-21004680,chr2:21004264-21004457,chr2:21001424-21003339 ukb23157_c2_b3_v1.vcf.gz ukb23157_c2_b4_v1.vcf.gz | bcftools view -Ou -T ^GRCh38_alldifficultregions.bed.gz | bcftools norm -Ou -m - -f GRCh38_reference_genome.fa | bcftools annotate -Ou --set-id '%CHROM\_%POS\_%REF\_%ALT' | bcftools view -Oz --threads 4 -i 'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(

In [44]:
!{dx_command}

job-GPG9KY8JqBj5gK412JXv3f8G
